In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-effectiveness/sample_submission.csv
/kaggle/input/feedback-prize-effectiveness/train.csv
/kaggle/input/feedback-prize-effectiveness/test.csv
/kaggle/input/feedback-prize-effectiveness/test/D72CB1C11673.txt


In [1]:
!pip install -U autogluon > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.


In [2]:
import pandas as pd
from autogluon.tabular import TabularPredictor
import os


In [11]:
data_path = '/kaggle/input/feedback-prize-effectiveness'
train_csv_path = os.path.join(data_path, 'train.csv')
train_df = pd.read_csv(train_csv_path)
print("Dữ liệu huấn luyện có", len(train_df), "hàng.")


Dữ liệu huấn luyện có 36765 hàng.


In [4]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(train_df, test_size = 0.2, random_state = 3, stratify = train_df['discourse_effectiveness'] )

In [5]:
label = 'discourse_effectiveness'

In [6]:
predictor = TabularPredictor(
    label=label,
    eval_metric='log_loss'
).fit(
    train_data = train_data,
    time_limit = 600
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250630_145108"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.81 GB / 31.35 GB (95.1%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions

[1000]	valid_set's multi_logloss: 0.652658


	-0.6438	 = Validation score   (-log_loss)
	70.81s	 = Training   runtime
	0.98s	 = Validation runtime
Fitting model: RandomForestGini ... Training model for up to 329.32s of the 329.32s of remaining time.
	-0.8224	 = Validation score   (-log_loss)
	168.01s	 = Training   runtime
	0.39s	 = Validation runtime
Fitting model: RandomForestEntr ... Training model for up to 160.16s of the 160.16s of remaining time.
	-0.8151	 = Validation score   (-log_loss)
	136.99s	 = Training   runtime
	0.35s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 21.98s of the 21.97s of remaining time.
	Many features detected (9055), dynamically setting 'colsample_bylevel' to 0.11043622308117063 to speed up training (Default = 1).
	To disable this functionality, explicitly specify 'colsample_bylevel' in the model hyperparameters.
	Ran out of time, early stopping on iteration 1.
	-1.0825	 = Validation score   (-log_loss)
	15.68s	 = Training   runtime
	0.51s	 = Validation runtime
Fitting mo

In [12]:
y_true = test_data[label]
y_pred = predictor.predict(test_data)
y_proba = predictor.predict_proba(test_data)


print("AutoGluon Evaluation:")
predictor.leaderboard(test_data, extra_metrics=['accuracy', 'log_loss', 'f1_macro', 'f1_weighted', 'roc_auc_ovr_macro', 'roc_auc_ovr_weighted'])



AutoGluon Evaluation:


,model,score_test,accuracy,log_loss,f1_macro,f1_weighted,roc_auc_ovr,roc_auc_ovr_weighted,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.607523,0.733850,-0.607523,0.639545,0.710914,0.850435,0.835072,-0.611240,log_loss,5.570127,1.374568,241.117007,0.004860,0.001756,0.292353,2,True,9
1,LightGBM,-0.630263,0.734938,-0.630263,0.650482,0.716398,0.845190,0.829035,-0.643773,log_loss,2.972107,0.975409,70.811756,2.972107,0.975409,70.811756,1,True,5
2,LightGBMXT,-0.726364,0.685299,-0.726364,0.582364,0.659501,0.796201,0.773060,-0.742729,log_loss,2.447841,0.568084,65.796991,2.447841,0.568084,65.796991,1,True,4
3,NeuralNetFastAI,-0.731691,0.694546,-0.731691,0.624737,0.684699,0.808582,0.794917,-0.725978,log_loss,0.287795,0.049090,33.025633,0.287795,0.049090,33.025633,1,True,3
4,RandomForestEntr,-0.801031,0.645451,-0.801031,0.456616,0.578240,0.758484,0.733711,-0.815078,log_loss,2.305365,0.348313,136.987265,2.305365,0.348313,136.987265,1,True,7
5,RandomForestGini,-0.805086,0.645723,-0.805086,0.462493,0.581448,0.753169,0.729884,-0.822409,log_loss,2.757589,0.393684,168.012482,2.757589,0.393684,168.012482,1,True,6
6,CatBoost,-1.082272,0.604651,-1.082272,0.357289,0.507299,0.550240,0.558968,-1.082466,log_loss,1.465542,0.505817,15.675976,1.465542,0.505817,15.675976,1,True,8
7,KNeighborsUnif,-2.341401,0.595131,-2.341401,0.445950,0.552527,0.629957,0.624845,-2.335141,log_loss,25.440251,7.240264,10.780032,25.440251,7.240264,10.780032,1,True,1
8,KNeighborsDist,-2.348156,0.577995,-2.348156,0.456147,0.551428,0.635092,0.627424,-2.352427,log_loss,25.011472,7.205302,10.486506,25.011472,7.205302,10.486506,1,True,2


In [12]:
import pandas as pd
from autogluon.tabular import TabularPredictor
import os

# --- Bước 1: Tải dữ liệu ---
# Đường dẫn tới dữ liệu trên môi trường Kaggle
data_path = '/kaggle/input/feedback-prize-effectiveness'
train_csv_path = os.path.join(data_path, 'train.csv')
test_csv_path = os.path.join(data_path, 'test.csv')
sample_submission_path = os.path.join(data_path, 'sample_submission.csv')

print("Đang tải dữ liệu...")
# Tải dữ liệu huấn luyện
train_df = pd.read_csv(train_csv_path)

# Tải dữ liệu kiểm tra
test_df = pd.read_csv(test_csv_path)

# Tải file submission mẫu để biết định dạng
sample_submission_df = pd.read_csv(sample_submission_path)

print("Tải dữ liệu thành công!")
print("Dữ liệu huấn luyện có", len(train_df), "hàng.")
print("Dữ liệu kiểm tra có", len(test_df), "hàng.")


# --- Bước 2: Chuẩn bị dữ liệu ---
# Biến mục tiêu (target) mà chúng ta cần dự đoán
label = 'discourse_effectiveness'

# AutoGluon có thể tự động xử lý các cột không cần thiết,
# nhưng để rõ ràng, chúng ta có thể chỉ chọn những cột quan trọng.
# Các feature chính là nội dung văn bản và loại của yếu tố lập luận.
# AutoGluon sẽ tự động xem 'discourse_text' là văn bản và 'discourse_type' là biến phân loại.
# Chúng ta không cần xóa các cột khác, AutoGluon sẽ tự bỏ qua chúng nếu không hữu ích.

# Hiển thị 5 dòng đầu của dữ liệu huấn luyện
print("\n5 dòng đầu của dữ liệu huấn luyện:")
print(train_df.head())


# --- Bước 3: Huấn luyện mô hình với AutoGluon ---
# Khởi tạo TabularPredictor
# - label: Tên cột mục tiêu cần dự đoán.
# - path: Thư mục để lưu lại các mô hình đã huấn luyện.
# - eval_metric: Thước đo để đánh giá mô hình, dùng 'log_loss' theo yêu cầu cuộc thi.
predictor = TabularPredictor(
    label=label,
    path='autogluon_models',
    eval_metric='log_loss'
)

# Bắt đầu huấn luyện
# - train_data: DataFrame chứa dữ liệu huấn luyện.
# - presets: Cấu hình có sẵn. 'best_quality' sẽ ưu tiên độ chính xác cao nhất,
#            nhưng tốn nhiều thời gian. Bạn có thể dùng 'high_quality' hoặc
#            'medium_quality_faster_train' để chạy nhanh hơn.
# - time_limit: Thời gian tối đa cho phép huấn luyện (tính bằng giây).
#               Rất quan trọng trong các cuộc thi có giới hạn thời gian chạy notebook.
#               Ví dụ: 2 giờ = 7200 giây.
#               Trên Kaggle, bạn có khoảng 9 tiếng (khoảng 32400 giây).
#               Chúng ta đặt 8 tiếng để có thời gian dự phòng.
time_limit_seconds = 1200

print("\nBắt đầu huấn luyện mô hình...")
predictor.fit(
    train_data=train_df,
    presets='best_quality', # Tối ưu cho độ chính xác
    time_limit=time_limit_seconds
)

print("Huấn luyện hoàn tất!")


# --- Bước 4: Xem lại kết quả và các mô hình đã huấn luyện ---
print("\nBảng xếp hạng các mô hình:")
leaderboard = predictor.leaderboard(train_df, silent=True)
print(leaderboard)


# --- Bước 5: Dự đoán trên tập kiểm tra (Test set) ---
print("\nBắt đầu dự đoán trên tập test...")
# Sử dụng predict_proba() để lấy xác suất cho mỗi lớp, vì cuộc thi yêu cầu log_loss.
predictions_proba = predictor.predict_proba(test_df)

print("Dự đoán hoàn tất!")
print("5 dòng đầu của kết quả dự đoán (xác suất):")
print(predictions_proba.head())


# --- Bước 6: Tạo tệp submission ---
print("\nTạo tệp submission...")
# Kết quả dự đoán có các cột là 'Ineffective', 'Adequate', 'Effective'.
# Chúng ta chỉ cần gán các giá trị này vào đúng cột trong file submission.
submission_df = test_df[['discourse_id']].copy()
submission_df[['Ineffective', 'Adequate', 'Effective']] = predictions_proba

# Lưu tệp submission
submission_df.to_csv('submission.csv', index=False)

print("\nTệp submission.csv đã được tạo thành công!")
print(submission_df.head())

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       27.91 GB / 31.35 GB (89.0%)
Disk Space Avail:   14.48 GB / 19.52 GB (74.2%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke

Đang tải dữ liệu...
Tải dữ liệu thành công!
Dữ liệu huấn luyện có 36765 hàng.
Dữ liệu kiểm tra có 10 hàng.

5 dòng đầu của dữ liệu huấn luyện:
   discourse_id      essay_id  \
0  0013cc385424  007ACE74B050   
1  9704a709b505  007ACE74B050   
2  c22adee811b6  007ACE74B050   
3  a10d361e54e4  007ACE74B050   
4  db3e453ec4e2  007ACE74B050   

                                                                                                                                                                                                                                                                                                                                                               discourse_text  \
0                                               Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the

	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-06-30 01:21:33,455	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
		Context path: "/kaggle/working/autogluon_models/ds_sub_fit/sub_fit_ho"
(_dystack pid=398) Running DyStack sub-fit ...
(_dystack pid=398) Beginning AutoGluon training ... Time limit = 295s
(_dystack pid=398) AutoGluon will save models to "/kaggle/working/autogluon_models/ds_sub_fit/sub_fit_ho"
(_dystack pid=398) Train Data Rows:    32680
(_dystack pid=398) Train Data Columns: 4
(_dystack pid=398) Label Column:       discourse_effectiveness
(_dystack pid=398) Problem Type:       multiclass
(_dystack pid=398) Preprocessing data ...
(_dystack pid=398) Train Data Class Count: 3
(_dystack pid=398) Using Feature Generators to preprocess the data ...
(_dystack pid=398) Fitting A

Huấn luyện hoàn tất!

Bảng xếp hạng các mô hình:
                    model  score_test  score_val eval_metric  pred_time_test  \
0   KNeighborsDist_BAG_L1   -0.000831  -2.299245    log_loss      161.872678   
1     WeightedEnsemble_L2   -0.304060  -0.666804    log_loss      165.793072   
2     WeightedEnsemble_L3   -0.344510  -0.645014    log_loss      337.348608   
3  NeuralNetFastAI_BAG_L2   -0.382564  -0.699641    log_loss      337.338576   
4  NeuralNetFastAI_BAG_L1   -0.401539  -0.700783    log_loss        3.916750   
5   KNeighborsUnif_BAG_L1   -0.605692  -2.294204    log_loss      161.701364   
6       LightGBMXT_BAG_L2   -0.934319  -0.943187    log_loss      348.527791   

   pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  \
0     149.071196   15.069632               161.872678              149.071196   
1     149.712203  141.729271                 0.003643                0.007970   
2     302.120903  354.791245                 0.010033              

In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    log_loss,
    f1_score,
    roc_auc_score,
)
from sklearn.preprocessing import LabelEncoder

In [4]:
TRAIN_CSV_PATH = '/kaggle/input/feedback-prize-effectiveness/train.csv'
TRAIN_ESSAYS_PATH = '/kaggle/input/feedback-prize-effectiveness/train/'

In [5]:
def create_dummy_data():
    """
    Creates dummy data files for local testing if they don't exist.
    This simulates the Kaggle environment.
    """
    print("Creating dummy data for local execution...")

    # Create dummy train.csv
    if not os.path.exists(TRAIN_CSV_PATH):
        train_data = {
            'discourse_id': [f'd{i}' for i in range(100)],
            'essay_id': [f'essay_{i % 10}' for i in range(100)],
            'discourse_text': [
                'This is a sample lead.', 'The author makes a claim.', 'Here is some evidence.',
                'This is a concluding statement.', 'Another position is taken here.'
            ] * 20,
            'discourse_type': ['Lead', 'Claim', 'Evidence', 'Concluding Statement', 'Position'] * 20,
            'discourse_effectiveness': ['Adequate', 'Effective', 'Ineffective', 'Adequate', 'Effective'] * 20
        }
        train_df = pd.DataFrame(train_data)
        train_df.to_csv(TRAIN_CSV_PATH, index=False)
        print(f"'{TRAIN_CSV_PATH}' created.")

    # Create dummy essay text files
    if not os.path.exists(TRAIN_ESSAYS_PATH):
        os.makedirs(TRAIN_ESSAYS_PATH)
        for i in range(10):
            essay_id = f'essay_{i}'
            essay_text = (
                f"This is the full text for essay {essay_id}. "
                "It contains various discourse elements that are analyzed. "
                "The student argues a point, provides evidence, and concludes the argument. "
                "The quality of these elements varies."
            )
            with open(os.path.join(TRAIN_ESSAYS_PATH, f'{essay_id}.txt'), 'w') as f:
                f.write(essay_text)
        print(f"'{TRAIN_ESSAYS_PATH}' directory and dummy essays created.")

In [6]:
def load_data(train_csv_path, essays_path):
    """
    Loads the training data from the CSV and merges it with the full essay texts.

    Args:
        train_csv_path (str): Path to the train.csv file.
        essays_path (str): Path to the directory containing essay .txt files.

    Returns:
        pandas.DataFrame: A DataFrame with the combined data.
    """
    print("Loading data...")
    # Load the main training data
    try:
        df = pd.read_csv(train_csv_path)
    except FileNotFoundError:
        print(f"Error: Training CSV not found at '{train_csv_path}'.")
        return None

    # Load essay texts into a dictionary
    essay_texts = {}
    for filename in os.listdir(essays_path):
        if filename.endswith('.txt'):
            essay_id = filename.split('.')[0]
            with open(os.path.join(essays_path, filename), 'r') as f:
                essay_texts[essay_id] = f.read()

    # Map the essay texts to the DataFrame
    df['essay_full_text'] = df['essay_id'].map(essay_texts)
    print("Data loading complete.")
    return df

In [7]:
def feature_engineering(df):
    """
    Creates a combined text feature for the model.

    Args:
        df (pandas.DataFrame): The input DataFrame.

    Returns:
        pandas.DataFrame: The DataFrame with the new 'full_context_text' feature.
    """
    print("Performing feature engineering...")
    # Combine the discourse text, type, and the full essay for context
    # Using a separator to give the model a hint about the different parts
    df['full_context_text'] = (
        df['discourse_type'] + ' [SEP] ' +
        df['discourse_text'] + ' [SEP] ' +
        df['essay_full_text']
    )
    print("Feature engineering complete.")
    return df

In [8]:
if __name__ == "__main__":
    # Create dummy files if they don't exist (for local runs)
    if not (os.path.exists(TRAIN_CSV_PATH) and os.path.exists(TRAIN_ESSAYS_PATH)):
        create_dummy_data()

    # 1. Load and Prepare Data
    train_df = load_data(TRAIN_CSV_PATH, TRAIN_ESSAYS_PATH)

    if train_df is not None:
        train_df = feature_engineering(train_df)

        # Define features (X) and target (y)
        X = train_df['full_context_text']
        y_raw = train_df['discourse_effectiveness']

        # 2. Encode Target Labels
        # The labels need to be converted from strings to integers (0, 1, 2)
        print("Encoding target labels...")
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y_raw)
        # The mapping will be: Adequate -> 0, Effective -> 1, Ineffective -> 2
        # We can see the classes with `label_encoder.classes_`
        class_names = label_encoder.classes_
        print(f"Labels encoded. Class mapping: {dict(zip(class_names, range(len(class_names))))}")


        # 3. Split Data into Training and Validation Sets
        print("Splitting data into training and validation sets (80/20 split)...")
        X_train, X_val, y_train, y_val = train_test_split(
            X, y,
            test_size=0.2,
            random_state=42,
            stratify=y  # Stratify to maintain class distribution in train/test splits
        )
        print(f"Training set size: {len(X_train)}")
        print(f"Validation set size: {len(X_val)}")

        # 4. Define the Model Pipeline
        # A pipeline makes it easy to chain preprocessing and modeling steps.
        # Step 1: TfidfVectorizer - Converts text to a matrix of TF-IDF features.
        # Step 2: LogisticRegression - A simple, effective baseline model for text classification.
        print("Defining the model pipeline...")
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(
                ngram_range=(1, 3), # Use unigrams, bigrams, and trigrams
                max_features=10000, # Limit the number of features to the top 10k
                stop_words='english'
            )),
            ('clf', LogisticRegression(
                solver='liblinear', # Good for smaller datasets
                random_state=42,
                C=1.0 # Regularization strength
            ))
        ])

        # 5. Train the Model
        print("Training the model...")
        pipeline.fit(X_train, y_train)
        print("Model training complete.")

        # 6. Make Predictions on the Validation Set
        print("Making predictions on the validation set...")
        y_pred = pipeline.predict(X_val)
        y_pred_proba = pipeline.predict_proba(X_val)
        print("Predictions complete.")

        # 7. Evaluate the Model
        print("\n--- Model Evaluation Results ---")

        # Calculate metrics
        accuracy = accuracy_score(y_val, y_pred)
        loss = log_loss(y_val, y_pred_proba)
        f1_macro = f1_score(y_val, y_pred, average='macro')
        f1_weighted = f1_score(y_val, y_pred, average='weighted')
        # For multiclass, roc_auc_score needs probabilities and multi_class='ovr'
        roc_auc_ovr = roc_auc_score(y_val, y_pred_proba, multi_class='ovr', average='macro')
        roc_auc_ovr_weighted = roc_auc_score(y_val, y_pred_proba, multi_class='ovr', average='weighted')

        # Create a DataFrame for a clean display of results
        results_df = pd.DataFrame({
            'Metric': [
                'Accuracy',
                'Log Loss',
                'F1 Score (Macro)',
                'F1 Score (Weighted)',
                'ROC AUC (OVR Macro)',
                'ROC AUC (OVR Weighted)'
            ],
            'Score': [
                accuracy,
                loss,
                f1_macro,
                f1_weighted,
                roc_auc_ovr,
                roc_auc_ovr_weighted
            ]
        })

        print(results_df.to_string(index=False))
        print("--------------------------------\n")

Loading data...
Data loading complete.
Performing feature engineering...
Feature engineering complete.
Encoding target labels...
Labels encoded. Class mapping: {'Adequate': 0, 'Effective': 1, 'Ineffective': 2}
Splitting data into training and validation sets (80/20 split)...
Training set size: 29412
Validation set size: 7353
Defining the model pipeline...
Training the model...
Model training complete.
Making predictions on the validation set...
Predictions complete.

--- Model Evaluation Results ---
                Metric    Score
              Accuracy 0.714538
              Log Loss 0.675059
      F1 Score (Macro) 0.600052
   F1 Score (Weighted) 0.681464
   ROC AUC (OVR Macro) 0.828139
ROC AUC (OVR Weighted) 0.809148
--------------------------------



In [2]:
import pandas as pd
import numpy as np
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    log_loss,
    f1_score,
    roc_auc_score,
)

# Suppress all warnings for cleaner output
warnings.filterwarnings('ignore')

# --- Configuration ---
# Define paths to the data files and directories.
# These paths are set up to work in a Kaggle-like environment.
TRAIN_CSV_PATH = '/kaggle/input/feedback-prize-effectiveness/train.csv'
TRAIN_DIR_PATH = '/kaggle/input/feedback-prize-effectiveness/train'

# Define the mapping for target labels
LABEL_MAPPING = {
    'Ineffective': 0,
    'Adequate': 1,
    'Effective': 2
}
# Reverse mapping for display if needed
REVERSE_LABEL_MAPPING = {v: k for k, v in LABEL_MAPPING.items()}

# --- Data Loading and Preparation ---

def create_dummy_data():
    """
    Creates dummy data files and directories for local testing if they don't exist.
    This simulates the Kaggle environment for development purposes.
    """
    print("Creating dummy data for local execution...")

    # Create parent directory for dummy data if it doesn't exist
    os.makedirs(os.path.dirname(TRAIN_CSV_PATH), exist_ok=True)
    os.makedirs(TRAIN_DIR_PATH, exist_ok=True)

    # Create dummy train.csv
    if not os.path.exists(TRAIN_CSV_PATH):
        train_data = {
            'discourse_id': ['d1', 'd2', 'd3', 'd4', 'd5', 'd6'],
            'essay_id': ['e1', 'e1', 'e2', 'e2', 'e3', 'e3'],
            'discourse_text': [
                "This is a strong claim about the topic.",
                "Evidence shows that this is true.",
                "Introduction to the essay.",
                "A counter-argument presented here.",
                "Concluding remarks.",
                "Another piece of evidence."
            ],
            'discourse_type': [
                'Claim', 'Evidence', 'Lead', 'Counterclaim', 'Concluding Statement', 'Evidence'
            ],
            'discourse_effectiveness': [
                'Effective', 'Adequate', 'Ineffective', 'Adequate', 'Effective', 'Adequate'
            ]
        }
        train_df = pd.DataFrame(train_data)
        train_df.to_csv(TRAIN_CSV_PATH, index=False)
        print(f"'{TRAIN_CSV_PATH}' created.")

    # Create dummy essay text files
    dummy_essays = {
        'e1': "This is the full essay text for essay e1. It contains a strong claim and supporting evidence.",
        'e2': "Essay e2 starts with an introduction and then presents a counter-argument.",
        'e3': "Essay e3 includes concluding remarks and additional evidence."
    }
    for essay_id, content in dummy_essays.items():
        essay_file_path = os.path.join(TRAIN_DIR_PATH, f"{essay_id}.txt")
        if not os.path.exists(essay_file_path):
            with open(essay_file_path, 'w') as f:
                f.write(content)
            print(f"'{essay_file_path}' created.")

def load_data(train_csv_path, train_dir_path):
    """
    Loads the training data from the CSV and the corresponding essay texts.

    Args:
        train_csv_path (str): Path to the train.csv file.
        train_dir_path (str): Path to the directory containing essay text files.

    Returns:
        pandas.DataFrame: A DataFrame with the loaded data, including essay texts.
    """
    print("Loading data...")
    try:
        df = pd.read_csv(train_csv_path)
    except FileNotFoundError:
        print(f"Error: Training CSV not found at '{train_csv_path}'.")
        return None

    # Load essay texts
    essay_texts = {}
    if os.path.exists(train_dir_path):
        for filename in os.listdir(train_dir_path):
            if filename.endswith('.txt'):
                essay_id = filename.replace('.txt', '')
                file_path = os.path.join(train_dir_path, filename)
                with open(file_path, 'r', encoding='utf-8') as f:
                    essay_texts[essay_id] = f.read()
    else:
        print(f"Warning: Essay directory not found at '{train_dir_path}'. "
              "Proceeding without full essay context.")

    # Map essay texts to the DataFrame
    df['full_essay_text'] = df['essay_id'].map(essay_texts)
    df['full_essay_text'] = df['full_essay_text'].fillna('') # Handle cases where essay text might be missing

    print("Data loading complete.")
    return df

# --- Main Execution Logic ---
if __name__ == "__main__":
    # Create dummy files if they don't exist (for local runs)
    if not os.path.exists(TRAIN_CSV_PATH) or not os.path.exists(TRAIN_DIR_PATH):
        create_dummy_data()

    # 1. Load Data
    train_df = load_data(TRAIN_CSV_PATH, TRAIN_DIR_PATH)

    if train_df is not None:
        # 2. Prepare Features and Target
        print("Preparing features and target...")
        # Map target labels to numerical values
        train_df['discourse_effectiveness_encoded'] = train_df['discourse_effectiveness'].map(LABEL_MAPPING)

        # Combine discourse_text and discourse_type for the main text feature
        # This allows TF-IDF to capture patterns related to both content and type
        X = train_df['discourse_type'] + " " + train_df['discourse_text']
        y = train_df['discourse_effectiveness_encoded']

        # Ensure all labels are present in the training set for consistent evaluation
        unique_labels = sorted(y.unique())
        if len(unique_labels) < len(LABEL_MAPPING):
            print(f"Warning: Not all target classes ({len(unique_labels)}/{len(LABEL_MAPPING)}) "
                  "are present in the dataset. This might affect metric calculations.")

        # 3. Split Data into Training and Validation Sets
        print("Splitting data into training and validation sets (80/20 split)...")
        X_train, X_val, y_train, y_val = train_test_split(
            X, y,
            test_size=0.2,
            random_state=42,
            stratify=y  # Stratify to maintain class distribution across splits
        )
        print(f"Training set size: {len(X_train)}")
        print(f"Validation set size: {len(X_val)}")

        # 4. Define the Model Pipeline
        print("Defining the model pipeline...")
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(
                ngram_range=(1, 2),  # Use unigrams and bigrams
                max_features=20000,  # Limit the number of features
                stop_words='english' # Remove common English stop words
            )),
            ('clf', LogisticRegression(
                solver='liblinear',  # Good for smaller datasets and L1/L2 regularization
                random_state=42,
                C=0.5,               # Regularization strength
                multi_class='ovr'    # One-vs-Rest strategy for multi-class classification
            ))
        ])

        # 5. Train the Model
        print("Training the model...")
        pipeline.fit(X_train, y_train)
        print("Model training complete.")

        # 6. Make Predictions on the Validation Set
        print("Making predictions on the validation set...")
        y_pred = pipeline.predict(X_val)
        y_pred_proba = pipeline.predict_proba(X_val) # Get probabilities for all classes
        print("Predictions complete.")

        # 7. Evaluate the Model
        print("\n--- Model Evaluation Results ---")
        # Calculate metrics
        accuracy = accuracy_score(y_val, y_pred)
        loss = log_loss(y_val, y_pred_proba, labels=unique_labels) # Ensure labels are passed to log_loss

        f1_macro = f1_score(y_val, y_pred, average='macro')
        f1_weighted = f1_score(y_val, y_pred, average='weighted')

        # ROC AUC for multi-class (OVR)
        # roc_auc_score requires probabilities for all classes (y_pred_proba)
        # and the true labels (y_val).
        # The 'labels' parameter should be explicitly passed to handle cases
        # where not all classes might be present in a small validation set,
        # but the model predicts probabilities for all expected classes.
        roc_auc_ovr = roc_auc_score(y_val, y_pred_proba, multi_class='ovr', average='macro', labels=unique_labels)
        roc_auc_ovr_weighted = roc_auc_score(y_val, y_pred_proba, multi_class='ovr', average='weighted', labels=unique_labels)


        # Create a DataFrame for a clean display of results
        results_df = pd.DataFrame({
            'Metric': [
                'Accuracy',
                'Log Loss',
                'F1 Score (Macro)',
                'F1 Score (Weighted)',
                'ROC AUC (OVR, Macro)',
                'ROC AUC (OVR, Weighted)'
            ],
            'Score': [
                accuracy,
                loss,
                f1_macro,
                f1_weighted,
                roc_auc_ovr,
                roc_auc_ovr_weighted
            ]
        })
        print(results_df.to_string(index=False))
        print("--------------------------------\n")

    else:
        print("Data loading failed. Exiting.")

Loading data...
Data loading complete.
Preparing features and target...
Splitting data into training and validation sets (80/20 split)...
Training set size: 29412
Validation set size: 7353
Defining the model pipeline...
Training the model...
Model training complete.
Making predictions on the validation set...
Predictions complete.

--- Model Evaluation Results ---
                 Metric    Score
               Accuracy 0.645451
               Log Loss 0.792616
       F1 Score (Macro) 0.487924
    F1 Score (Weighted) 0.593754
   ROC AUC (OVR, Macro) 0.763501
ROC AUC (OVR, Weighted) 0.738914
--------------------------------

